## 支持数据分析的工具

In [1]:
import pandas as pd
from illufly.agent import Dataset, ChatQwen
from illufly.tools import create_python_code_tool
from illufly.io import log

data = {
    "考试成绩": Dataset(
        df=pd.DataFrame({
            "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
            "语文": [90, 80, 105, 110],
            "数学": [135, 110, 120, 90]
        }),
        desc="考试成绩"
    )
}
mytool = create_python_code_tool(data, ChatQwen())

# mytool
for x in mytool.func({"question": "赖国良成绩如何？"}):
    print(x)

```
python


def main():
    #
 从数据集中筛选出赖国
良的成绩
    lai_guo
_liang_scores = data['考试成绩
'].df[data['考试成绩'].df
['姓名'] == '赖国良
']
    # 返回赖国良的成绩

    return lai_guo_li
ang_scores[['语文', '数学']].
to_dict(orient='records')

```

```python
def main():
    # 从数据集中筛选出赖国良的成绩
    lai_guo_liang_scores = data['考试成绩'].df[data['考试成绩'].df['姓名'] == '赖国良']
    # 返回赖国良的成绩
    return lai_guo_liang_scores[['语文', '数学']].to_dict(orient='records')

```
语文: 105, 数学: 120


## 绑定 + 数据分析

### 多轮对话

In [1]:
from illufly.agent import ChatQwen, FakeLLM
from illufly.io import log, alog

import pandas as pd

a = ChatQwen()

a.state.add_dataset(
    "考试成绩",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "语文": [90, 80, 105, 110],
        "数学": [135, 110, 120, 90]
    })
)

In [4]:
await alog(a, "请使用工具查询，看看谁的语文考试成绩最好？", new_chat=True)

```python
def main():
    # 加载考试成绩数据集
    exam_scores = data['考试成绩'].df
    
    # 确定语文成绩最好的学生
    best_chinese_student = exam_scores.loc[exam_scores['语文'].idxmax()]['姓名']
    
    return best_chinese_student

main()
```

语文考试成绩最好的是吴红兵。



'{"0": {"index": 0, "id": "call_31b1904bb11d40eaaab906", "type": "function", "function": {"name": "python_code", "arguments": "{\\"question\\": \\"谁的语文考试成绩最好？\\"}"}}}语文考试成绩最好的是吴红兵。'

In [5]:
log(a, "看看他的总分第几名？")

```python
import pandas as pd

def main():
    global data
    # Load the exam scores dataset
    exam_scores_df = data['考试成绩'].df
    
    # Add a column for total score
    exam_scores_df['总分'] = exam_scores_df['语文'] + exam_scores_df['数学']
    
    # Sort the dataframe by total score in descending order
    sorted_scores_df = exam_scores_df.sort_values(by='总分', ascending=False)
    
    # Get Wu Hongbing's total score
    wu_hongbing_total = sorted_scores_df.loc[sorted_scores_df['姓名'] == '吴红兵', '总分'].iloc[0]
    
    # Get Wu Hongbing's rank
    wu_hongbing_rank = sorted_scores_df['总分'].eq(wu_hongbing_total).sum()
    
    return wu_hongbing_rank

main()
```

吴红兵的总分排名是第一名。



'{"0": {"index": 0, "id": "call_93556a2dfeb44eb282a993", "type": "function", "function": {"name": "python_code", "arguments": "{\\"question\\": \\"吴红兵的总分排名是多少？\\"}"}}}吴红兵的总分排名是第一名。'

In [6]:
log(a, "再帮我看看薛一凡的详细情况和名次")

```python
import pandas as pd

def rank_student(data, name):
    df = data['考试成绩'].df
    # 添加总分列（如果尚未添加）
    if '总分' not in df.columns:
        df['总分'] = df['语文'] + df['数学']
    # 按总分降序排序并添加排名列
    df_ranked = df.sort_values(by='总分', ascending=False).reset_index(drop=True)
    df_ranked['排名'] = df_ranked.index + 1
    # 查询指定学生的成绩信息及排名
    student_info = df_ranked[df_ranked['姓名'] == name]
    return student_info.to_dict(orient='records')

def main():
    result = rank_student(data, '薛一凡')
    return result
```

薛一凡的详细成绩信息如下：
- 语文: 90 分
- 数学: 135 分
总分为 225 分，排名是第一名。



'{"0": {"index": 0, "id": "call_7e5f7b34efe84b1681f560", "type": "function", "function": {"name": "python_code", "arguments": "{\\"question\\": \\"查询薛一凡的详细成绩信息及排名\\"}"}}}薛一凡的详细成绩信息如下：\n- 语文: 90 分\n- 数学: 135 分\n总分为 225 分，排名是第一名。'

### 单轮写作

In [8]:
from illufly.llm import qwen, fake_llm
from illufly.agent import Agent
from illufly.io import log
import pandas as pd

a = Agent(qwen)

a.add_dataset(
    "中秋名单",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "月饼数量": [2, 2, 1, 1]
    })
)

a.add_dataset(
    "考试成绩",
    pd.DataFrame({
        "姓名": ["薛一凡", "肖一笑", "赖国良", "吴红兵"],
        "语文": [90, 80, 105, 110],
        "数学": [135, 110, 120, 90]
    })
)

log(a, "请查询工具，针对这几个孩子的实际考试成绩，帮我写一份报告", template="OUTLINE")

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_f35cc3756f604157a2223a", "type": "function", "function": {"name": "python_code", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"question\": \"查询"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "几个孩子的实际考试成绩\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_f35cc3756f604157a2223a", "type": "function", "function": {"name": "python_code", "arguments": "{\"question\": \"查询几个孩子的实际考试成绩\"}"}}}
```python
def main():
    # 由于问题没有具体说明要查询哪些孩子的成绩以及需要什么样的处理，
    # 这里我们做一个基本的假设：查询所有孩子的语文和数学成绩。
    # 如果有具体需求，请提供更详细的问题描述。

    # 使用pandas库读取考试成绩数据集
    exam_scores = data['考试成绩'].df
    
    # 返回所有孩子的语文和数学成绩
    return exam_scores[['姓名', '语文', '数学']]
```

[TOOL_RESP_FINAL] 
| 姓名   |   语文 |   数学 |
|

'# 学生考试成绩分析报告\n\n## 一、成绩概览\n\n### 成绩统计\n\n<OUTLINE>\n扩写摘要: 对学生的语文与数学成绩进行整体概括，列出平均分、最高分、最低分。\n\n扩写要求:\n- 计算并列举所有学生的语文和数学成绩的平均分。\n- 高出平均分最多的学生及其分数。\n- 低于平均分最多的学生及其分数。\n- 预估字数：150字\n</OUTLINE>\n\n## 二、个体成绩分析\n\n### 薛一凡\n\n<OUTLINE>\n扩写摘要: 分析薛一凡同学的成绩表现，与班级平均成绩对比。\n\n扩写要求:\n- 对比薛一凡的语文和数学成绩与班级相应科目的平均成绩。\n- 点评薛一凡成绩的优势科目及提升空间。\n- 预估字数：100字\n</OUTLINE>\n\n### 肖一笑\n\n<OUTLINE>\n扩写摘要: 评估肖一笑同学的学习情况，提出针对性建议。\n\n扩写要求:\n- 指出肖一笑成绩相对薄弱的科目。\n- 给出提高该科目成绩的具体建议。\n- 预估字数：100字\n</OUTLINE>\n\n### 赖国良\n\n<OUTLINE>\n扩写摘要: 分析赖国良的成绩分布，探讨其学习策略的有效性。\n\n扩写要求:\n- 分析赖国良在两门科目上的表现是否均衡。\n- 探讨赖国良维持或提升成绩可能采取的策略。\n- 预估字数：100字\n</OUTLINE>\n\n### 吴红兵\n\n<OUTLINE>\n扩写摘要: 评价吴红兵的成绩波动，提出平衡发展的重要性。\n\n扩写要求:\n- 说明吴红兵成绩的显著差异（高分与低分科目）。\n- 强调平衡各科学习的必要性，并提供策略。\n- 预估字数：100字\n</OUTLINE>\n\n## 三、总体建议与总结\n\n<OUTLINE>\n扩写摘要: 综合所有学生的表现，提出班级整体提升学习效率的方法。\n\n扩写要求:\n- 总结学生们的共性问题和个别亮点。\n- 提出适用于全班的一般性学习策略和方法。\n- 预估字数：150字\n</OUTLINE>'